# Function Optimum Collection for Mf2 Functions

The best way to (objectively) evaluate optimization performance relies on knowing the location of the global optimum $x^*$
This notebook is intended as a collection space for the (literature) research on finding the global optima for the various functions in the `mf2` collection.

For each function, the following information will be listed:
 - mathematical definition of the function
 - typical evaluation bounds within which we want to know the optimum
 - location $x^*$ of the global optimum (or optima if multiple exist) within
 - optimum function value $f(x^*)$
 - source of the optimum location, i.e.
   - mathematical derivation of the optimum -OR-
   - reference listing a known optimum if not (easily) derivable -OR-
   - procedure describing the numerical search method used to establish $x^*$

## Full Table

Here is the full table, composed of entries from each function as specified below:

| Name              | Bounds                     | $$x^*$$                 | $$f(x^*)$$            | Source |
| :---------------- | :------------------------- | ----------------------: | --------------------: | ------ |
| Forrester         | $$x \in [0,1]$$            | 0.757248757841856       | -6.0207400557670825   | visual inspection + [WolframAlpha][1]  |
| Bohachevsky       | $$x,y \in [-5,5]$$         | (0, 0)                  | 0                     | [sfu.ca/~ssurjano/boha][2] |
| Booth             | $$x,y\in[-10,10]$$         | (1,3)                   | 0                     | [Wikipedia][wiki_test_functions] |
| Branin            | $$x\in[-5,10],y\in[0,15]$$ | (-3.786088705282203, 15) | -333.9160343522788 | scipy.minimize starting from (-$\pi$, 12.275) |
| Currin            | $$x,y\in[0,1]$$            | (0.21666..., 0)         | 13.798722044728434[<sup>1</sup>](#fn1) | visual + [WolframAlpha][wolframalpha_currin] for y=0 |
| Himmelblau        | $$x,y\in[-4,4]$$           |(3,2),(-2.805118,3.131312),(-3.779310,-3.283286),(3.584428,-1.848126)|0|[Wikipedia][wiki_test_functions] |
| Six Hump Camelback| $$x,y\in[-2,2]$$           | (0.0898,-0.7126),(-0.0898,0.7126)| -1.0316| [sfu.ca/~ssurjano/camel6][4] |
| Park 91 a   | $$x_i\in[0,1) \forall i$$ | (10$^{-8}$, 0, 0, 0)[<sup>2</sup>](#fn2) | 2.718281828459045 * 10$^{-8}$ | repeated scipy.optimize |
| Park 91 B   | $$x_i\in[0,1) \forall i$$ | (0,0,0,0)[<sup>3</sup>](#fn3) | $$\frac{2}{3}$$ | Simple derivation |
| Hartmann 6D | $$x_i\in[0.1, 1]\forall i$$ | ~(0.2017, 0.1500, 0.4769, 0.2753, 0.3117, 0.6573) | -3.0424577378430477[<sup>4</sup>](#fn4) | [sfu.ca/~ssurjano/hart6][ssurhart6] |
| Borehole    | l_bound = [0.05,    100,  63_070,   990, 63.1, 700, 1_120,  9_855],<br> u_bound = [0.15, 50_000, 115_600, 1_110,  116, 820, 1_680, 12_045] | (5e-2, 5e4, 6.307e4, 9.9e2, 6.31e1, 8.2e2, 1.68e3, 9.855e3) | 7.819676328755232 | repeated scipy.optimize |




<span id="fn1"><sup>1</sup> this is the global **maximum** within the given bounds</span><br>
<span id="fn2"><sup>2</sup> Because of the $\frac{x_4}{x_1^2}$ in the function definition, the lower bound for $x_1$ is restricted to $10^{-8}$ to prevent divide-by-zero errors</span><br>
<span id="fn3"><sup>3</sup> $x_4$ is irrelevant once $x_3$ converges to 0</span><br>
<span id="fn4"><sup>4</sup>[sfu.ca/~ssurjano/hart6][ssurhart6] reports the optimum $f(x^*) = -3.32237$ for the unscaled version. Location of the optimum remains unchanged when scaling  </span>


[1]: https://www.wolframalpha.com/input/?i=derivative+%286x-2%29%5E2*sin%2812x-4%29+%3D+0
[2]: http://www.sfu.ca/~ssurjano/boha.html
[4]: http://www.sfu.ca/~ssurjano/camel6.html
[wolframalpha_currin]: https://www.wolframalpha.com/input/?i=derivative+%282300x%5E3+%2B+1900x%5E2+%2B+2092x+%2B+60%29+%2F+%28100x%5E3+%2B+500x%5E2+%2B+4x+%2B+20%29+%3D+0
[wiki_test_functions]: https://en.wikipedia.org/wiki/Test_functions_for_optimization
[ssurhart6]: http://www.sfu.ca/~ssurjano/hart6.html

## Global imports

In [ ]:
import mf2
import numpy as np
from scipy.optimize import minimize

np.set_printoptions(floatmode='maxprec', precision=99)

def get_start_positions(bounds, n_steps_per_dim=5):
    """Create a grid of starting points scaled to the
    given bounds
    """
    l_bound, u_bound = bounds
    x = np.linspace(0,1,n_steps_per_dim)
    bound_range = u_bound - l_bound

    coords = np.meshgrid(*[x for _ in u_bound])
    coords = np.stack([c.flatten() for c in coords], axis=1)

    return (coords * bound_range) + l_bound


def grid_optimize_mff(mf_func, n_steps_per_dim=5):
    """Run scipy.optimize repeatedly from a grid of starting
    positions to roughly determine the global optimum
    """
    func = mf_func.high
    bounds = mf_func.bounds

    results = [
        minimize(func, x_start, bounds=bounds.T)
        for x_start in get_start_positions(bounds, n_steps_per_dim)
    ]

    min_result = min(results, key=lambda x: x.fun)    
    print(f'Function:     {mf_func.name}')
    print(f'best x:       {min_result.x}')
    print(f'minimum f(x): {min_result.fun}')


def repeat_optimize_mff(mf_func, x_opt, repeat=10):
    """Repeatedly restart scipy.optimize from a previous
    optimal start position to refine all decimals of the
    global optimum
    """
    func = mf_func.high
    bounds = mf_func.bounds
    
    for _ in range(repeat):
        result = minimize(func, x_opt, bounds=bounds.T)
        x_opt = result.x
        
    print(f'Function:     {mf_func.name}')
    print(f'best x:       {result.x}')
    print(f'minimum f(x): {result.fun}')


## 1D functions

### Forrester

Forrester high-fidelity: $f_h(x) = (6x-2)^2 \sin(12x-4)$

| Name      | Bounds               | $$x^*$$             | $$f(x^*)$$            | Source |
| --------- | -------------------- | ------------------- | --------------------- | ------ |
| Forrester | $$x \in [0,1]$$      | ~ 0.757248757841856 | ~ -6.0207400557670825 | visual inspection + [WolframAlpha][1]  |

[1]: https://www.wolframalpha.com/input/?i=derivative+%286x-2%29%5E2*sin%2812x-4%29+%3D+0

In [ ]:
mf2.forrester.high(0.757248757841856)

In [ ]:
repeat_optimize_mff(mf2.forrester, 0.757248757841856)

## 2D functions

### Bohachevsky

Bohachevsky high-fidelity: $ f_h(x) = x_1^2 + 2x_2^2 - 0.3\cos(3\pi x_1) - 0.4\cos(4\pi x_2) + 0.7$

| Name        | Bounds           | $$x^*$$   | $$f(x^*)$$ | Source                |
| ----------- | ---------------- | --------- | ---------- | --------------------- |
| Bohachevsky | $$x \in [-5,5]$$ | (0, 0)    | 0          | [sfu.ca/~ssurjano][2] |


[2]: http://www.sfu.ca/~ssurjano/boha.html

In [ ]:
mf2.bohachevsky.high([0,0])

In [ ]:
repeat_optimize_mff(mf2.bohachevsky, [0,0])

### Booth

Booth high-fidelity: $ f_h(x) = (x_1 + 2x_2 - 7)^2 + (2x_1 + x_2 - 5)^2 $

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Booth       | $$x,y\in[-10,10]$$ | $$(1,3)$$           | 0                     | [Wikipedia][wiki_test_functions] |

[wiki_test_functions]: https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [ ]:
mf2.booth.high([1,3])

In [ ]:
repeat_optimize_mff(mf2.booth, [1,3])

### Branin

Branin base: $f_b(x) = \Bigg(x_2 - (5.1\dfrac{x_1^2}{4\pi^2}) + \dfrac{5x_1}{\pi} - 6\Bigg)^2 + \Bigg(10\cos(x_1) (1 - \dfrac{1}{8\pi}\Bigg) + 10 $

The branin 'base' function has three optima: $x^*$ = (-$\pi$, 12.275), ($\pi$, 2.275), (9.42478, 2.475), with $f(x^*)$ = 0.397887 (source: [sfu.ca/~ssurjano/branin][3])

Branin high-fidelity: $ f_h(x) = f_b(x) - 22.5x_2 $ (from [Dong et al. 2015][5])

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Branin      | $$x_1\in[-5,10],x_2\in[0,15]$$ | (-3.786088705282203, 15) | -333.9160343522788 | scipy.minimize starting from (-$\pi$, 12.275) |

[3]: http://www.sfu.ca/~ssurjano/branin.html
[5]: https://doi.org/10.1007/s00158-014-1213-9

In [ ]:
branin_base_optima = [
    [ -np.pi, 12.275],
    [  np.pi,  2.275],
    [9.42478,  2.475],
]
print(f'branin.high(branin_base_optima) = {mf2.branin.high(branin_base_optima)}')

result = minimize(mf2.branin.high, branin_base_optima[0], bounds=mf2.branin.bounds.T)
print(result.x, result.fun)

In [ ]:
for opt in branin_base_optima:
    repeat_optimize_mff(mf2.branin, opt, repeat=20)

In [ ]:
repeat_optimize_mff(mf2.branin, [-3.786088705282203, 15])

### Currin

Currin: $f_h(x) = \Bigg( 1 - \exp(-\dfrac{1}{2x_2})\Bigg) \dfrac{2300x_1^3 + 1900x_1^2 + 2092x_1 + 60}{100x_1^3 + 500x_1^2 + 4x_1 + 20}$

Since there is no interaction between $x_1$ and $x_2$, we can simplify the search for the optimum by considering each variable separately.

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Currin      | $$x \in [0,1]$$    | (0.21666..., 0)       | 13.798722044728434[<sup>1</sup>](#fn1)    | visual + [WolframAlpha][wolframalpha_currin] for y=0 |

<span id="fn1"><sup>1</sup>: this is the global **maximum** within the given bounds</span>

[wolframalpha_currin]: https://www.wolframalpha.com/input/?i=derivative+%282300x%5E3+%2B+1900x%5E2+%2B+2092x+%2B+60%29+%2F+%28100x%5E3+%2B+500x%5E2+%2B+4x+%2B+20%29+%3D+0

In [ ]:
mf2.currin.high([0.21666666666666666666, 0])

In [ ]:
repeat_optimize_mff(mf2.invert(mf2.currin), [0.21666666666666666666, 0])

### Himmelblau

Himmelblau high-fidelity: $f_h(x) = (x_1^2 + x_2 - 11)^2 + (x_2^2 + x_1 - 7)^2$

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Himmelblau  | $$x\in[-4,4]$$|(3,2),(-2.805118,3.131312),(-3.779310,-3.283286),(3.584428,-1.848126)|0|[Wikipedia][wiki_test_functions] |


[wiki_test_functions]: https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [ ]:
himmelblau_optima = [
    [ 3,         2       ],
    [-2.805118,  3.131312],
    [-3.779310, -3.283286],
    [ 3.584428, -1.848126],
]
print(f'himmelblau.high(himmelblau_optima) = {mf2.himmelblau.high(himmelblau_optima)}')

In [ ]:
for opt in himmelblau_optima:
    repeat_optimize_mff(mf2.himmelblau, opt)

### Six-Hump Camelback

Six-Hump Camelback high-fidelity: $f_h(x) = 4x_1^2 - 2.1x_1^4 + \dfrac{x_1^6}{3} + x_1x_2 - 4x_2^2 + 4x_2^4 $

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Six Hump Camelback | $$x\in[-2,2]$$| (0.0898,-0.7126), (-0.0898,0.7126) | -1.0316 | [sfu.ca/~ssurjano/camel6][4] |

[4]: http://www.sfu.ca/~ssurjano/camel6.html

In [ ]:
sixhump_optima = [
    [ 0.0898, -0.7126],
    [-0.0898,  0.7126],
]
print(f'six_hump_camelback.high(sixhump_optima) = {mf2.six_hump_camelback.high(sixhump_optima)}')

In [ ]:
for opt in sixhump_optima:
    repeat_optimize_mff(mf2.six_hump_camelback, opt)

## 4D functions

### Park 91 A

Park 91A high-fidelity: $f_h(x) = \dfrac{x_1}{2} \Bigg(\sqrt{1 + (x_2 + x_3^2) * \dfrac{x_4}{x_1^2}} - 1\Bigg) + (x_1 + 3x_4)\exp(1 + \sin(x_3))$

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Park 91 a   | $$x_i\in[0,1) \forall i$$[<sup>3</sup>](#fn3) | (10$^{-8}$, 0, 0, 0) | 2.718281828459045 * 10$^{-8}$ | repeated scipy.optimize |


<span id="fn3"><sup>3</sup>: Because of the $\frac{x_4}{x_1^2}$ in the function definition, the lower bound for $x_1$ is restricted to $10^{-8}$ to prevent divide-by-zero errors</span>

In [ ]:
print(mf2.park91a.high([1e-8,0,0,0]))

result = minimize(mf2.park91a.high, [.5, .5, .5, .5], bounds=mf2.park91a.bounds.T)
print(result.x, result.fun)

In [ ]:
grid_optimize_mff(mf2.park91a, n_steps_per_dim=4)

In [ ]:
repeat_optimize_mff(mf2.park91a, [1e-8, 0, 0, 0])

#### Better inverted? Nope...

In [ ]:
grid_optimize_mff(mf2.invert(mf2.park91a), n_steps_per_dim=4)

### Park 91 B

Park 91B high-fidelity: $f_h(x) = \dfrac{2}{3}\exp(x_1 + x_2) - x_4\sin(x_3) + x_3$

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Park 91 B   | $$x_i\in[0,1) \forall i$$ | (0,0,0,0)[<sup>3</sup>](#fn3) | $$\dfrac{2}{3}$$ | Simple derivation |

<span id="fn3"><sup>3</sup> $x_4$ is irrelevant once $x_3$ converges to 0</span>

In [ ]:
mf2.park91b.high([0,0,0,0])

In [ ]:
repeat_optimize_mff(mf2.park91b, [0, 0, 0, 0])

#### Derivation

The Park91b function quite simply consists of two parts with independent $x_i$ variables: the exponent part with $x_1, x_2$, and the sine part with $x_3, x_4$.

For the exponent part, the minimum within the range $[0, 1)$ is achieved at $\dfrac{2}{3}\exp(0) = \dfrac{2}{3}$, hence $x_1 + x_2 = 0$ and therefore $x_1 = x_2 = 0$.

Initially assuming $x_4=1$ for the sine part, $x_3$ increases faster than $\sin(x_3)$, so the minimum is reached at $x_3=0$. Once $x_3=0$, $x_4\sin(0)=0$ for any value of $x_4$, so it is irrelevant.

## 6D functions


### Hartmann6

Hartmann6 high-fidelity: $f_h(x) = -\dfrac{1}{1.94}\Bigg( 2.58 + \sum^4_{i=1}\alpha_i \exp\Bigg( -\sum^6_{j=1} A_{ij}(x_j - P_{ij})^2 \Bigg) \Bigg)$, where,

```
\alpha = [1.0, 1.2, 3.0, 3.2].T
A = [
    [10.00,  3.0, 17.00,  3.5,  1.7,  8],
    [ 0.05, 10.0, 17.00,  0.1,  8.0, 14],
    [ 3.00,  3.5,  1.70, 10.0, 17.0,  8],
    [17.00,  8.0,  0.05, 10.0,  0.1, 14],
]
P = [
    [.1312, .1696, .5569, .0124, .8283, .5886],
    [.2329, .4135, .8307, .3736, .1004, .9991],
    [.2348, .1451, .3522, .2883, .3047, .6650],
    [.4047, .8828, .8732, .5743, .1091, .0381],
]
```

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Hartmann 6D | $$x_i\in[0.1, 1]\forall i$$ | ~(0.2017, 0.1500, 0.4769, 0.2753, 0.3117, 0.6573) | -3.0424577378430477[<sup>4</sup>](#fn4) | [sfu.ca/~ssurjano/hart6][ssurhart6] |

<span id="fn4"><sup>4</sup>[sfu.ca/~ssurjano/hart6][ssurhart6] reports the optimum $f(x^*) = -3.32237$ for the unscaled version. Location of the optimum remains unchanged when scaling  </span>

[ssurhart6]: http://www.sfu.ca/~ssurjano/hart6.html

In [ ]:
# Unscaled optimum, as reported by http://www.sfu.ca/~ssurjano/hart6.html
fx = mf2.hartmann6.high([0.20169, 0.150011, 0.476874, 0.275332, 0.311652, 0.6573])
print(fx)

In [ ]:
print((fx / (-1/1.94)) - 2.58)
print((3.3223680113913385 + 2.58) * (-1/1.94))


In [ ]:
grid_optimize_mff(mf2.hartmann6, n_steps_per_dim=4)

In [ ]:
repeat_optimize_mff(mf2.hartmann6, [0.2016895136059688, 0.15001068960347907, 0.4768739646943634, 0.2753324238312406, 0.3116516144713434, 0.6573005290552848])

In [ ]:
# Confirming correctness of optimum before and after scaling
x_opt = [0.2016895136059688, 0.15001068960347907, 0.4768739646943634, 0.2753324238312406, 0.3116516144713434, 0.6573005290552848]
f_opt = mf2.hartmann6.high(x_opt)[0]
print(f'{f_opt = }')
print(f'{(-f_opt * 1.94) - 2.58 = }')
print(f'{(-1/1.94) * (2.58 + 3.3223) = }')

## 8D functions

### Borehole

Borehole high-fidelity: $f_h(x) = \dfrac{A*T_u*(H_u - H_l)}{\Bigg(\log(\frac{r}{r_w}) * (B + \dfrac{2L*T_u}{\log(\frac{r}{r_w}) * r_w^2 * K_w} + \dfrac{T_u}{T_l}\Bigg)}$

| Name        | Bounds             | $$x^*$$             | $$f(x^*)$$            | Source |
| :---------- | :----------------- | ------------------: | --------------------: | ------ |
| Borehole    | l_bound = [0.05,    100,  63_070,   990, 63.1, 700, 1_120,  9_855], u_bound = [0.15, 50_000, 115_600, 1_110,  116, 820, 1_680, 12_045] | (5e-2, 5e4, 6.307e4, 9.9e2, 6.31e1, 8.2e2, 1.68e3, 9.855e3) | 7.819676328755232 | repeated scipy.optimize |

In [ ]:
grid_optimize_mff(mf2.borehole, n_steps_per_dim=3)

In [ ]:
repeat_optimize_mff(mf2.borehole, [5e-2, 5e4, 6.307e4, 9.9e2, 6.31e1, 8.2e2, 1.68e3, 9.855e3])